# Light models options

## Content

- CARIBU
    - sun algo
    - virtual sensors
    - autres parametre
- RATP
    - distribution angles foliaires
    - tesselation sur la grille
    - autre paramètres
    
## Introduction

During our use of lightvegemanager, we added special features for each known light models. This notebook gives you a small introduction to them.

The parameters of those features are stored in a dict.

In [1]:
import os
from lightvegemanager.LVM import LightVegeManager
from pgljupyter import SceneWidget
from lightvegemanager.trianglesmesh import random_triangle_generator

## CARIBU

This is the complete parameters you can provide with CARIBU: 

```python
caribu_args = {
                    "sun algo" : "ratp",
                    "sun algo" : "caribu",

                    "caribu opt" : {
                                    band0 = (reflectance, transmittance),
                                    band1 = (reflectance, transmittance),
                                    ...
                                    },
                    "debug" : bool,
                    "soil mesh" : bool,
                    "sensors" : ["grid", dxyz, nxyz, orig, vtkpath, "vtk"]
                }
```

### Computing the sun position

In order to compute the sun position, you can use either the algorithm from RATP or CARIBU. The (x, y, z) output is formatted in CARIBU format.

In [2]:
caribu_args = { "sun algo" : "caribu" }

lighting = LightVegeManager(lightmodel="caribu", lightmodel_parameters=caribu_args)
lighting.build(geometry=[(0., 0., 0.), (1., 0., 0.), (1., 1., 1.)])
energy = 500.
hour = 15
day = 264
lighting.run(energy=energy, hour=hour, day=day)

sun_caribu = lighting.sun
print(sun_caribu)

(0.33506553253259913, -0.8798617206271511, -0.3370080733212115)


In [3]:
caribu_args = { "sun algo" : "ratp" }

lighting = LightVegeManager(lightmodel="caribu", lightmodel_parameters=caribu_args)
lighting.build(geometry=[(0., 0., 0.), (1., 0., 0.), (1., 1., 1.)])
energy = 500.
hour = 15
day = 264
lighting.run(energy=energy, hour=hour, day=day)

sun_ratp = lighting.sun
print(sun_ratp)

(0.33241183146897624, -0.8800565622452903, -0.3391206592769639)


In [4]:
dist = (sum([ (x-y)**2 for x,y in zip(sun_ratp, sun_caribu) ])) ** (1/2)
print("euclidean dist = ",dist," m")

euclidean dist =  0.003397515564596359  m


### Grid of virtual sensors

If you can to match a grid of voxels, you can generate a set of virtual sensors following a 3D grid. You need to precise the dimension of the grid:
- dxyz: `[dx, dy, dz]` size of one voxel
- nxyz: `[nx, ny, nz]` number of voxels on each xyz axis
- orig: `[0x, 0y, 0z]` origin point of the grid

Optionnaly, you can write a geometric visualisation of the sensors in VTK format. You need to provide the file path and the flag `"vtk"`.

In [8]:
# grid dimensions
dxyz = [1.] * 3
nxyz = [5, 5, 7]
orig = [0.] * 3

In [9]:
# random triangles
nb_triangles = 50
spheresize = (1., 0.3) # vertices of triangles are the sphere surface
triangles = []
for i in range(nb_triangles):
    triangles.append(random_triangle_generator(worldsize=(0., 5.), spheresize=spheresize))

In [10]:
caribu_args = { "sensors" : ["grid", dxyz, nxyz, orig] }

lighting = LightVegeManager(lightmodel="caribu", lightmodel_parameters=caribu_args, environment={"infinite":True})
lighting.build(geometry=triangles)

You can visualize the grid of sensors with plantGL through the method `plantGL_sensors`

In [11]:
SceneWidget(lighting.plantGL_sensors(), 
            position=(-2.5, -2.5, 0.0), 
            size_display=(600, 400), 
            plane=True, 
            size_world = 10, 
            axes_helper=True)

SceneWidget(axes_helper=True, scenes=[{'id': '8cwhGR0RpCR4UmXwXjqEDf9BQ', 'data': b'x\xda\x85\x9ai\x83\x16G\x1…

The lighting results are stored in `sensors_outputs`

In [12]:
energy = 500.
hour = 15
day = 264
lighting.run(energy=energy, hour=hour, day=day)

print(lighting.sensors_outputs)

{'par': {0: 0.5620043191303375, 1: 0.5618584729038281, 2: 0.60389586895853, 3: 0.5994287825556907, 4: 0.6003338819673372, 5: 0.9892227473385927, 6: 0.5420517619926417, 7: 0.552905696886898, 8: 0.5743983291679143, 9: 0.5231573288803778, 10: 0.7745655468808682, 11: 0.9950071883035141, 12: 0.49672566375401617, 13: 0.5718796592189763, 14: 0.48319737010376, 15: 0.6195291102018295, 16: 0.9035769753518178, 17: 0.997274929628352, 18: 0.47939433586083885, 19: 0.5249417244166038, 20: 0.5515631860087062, 21: 0.6541147586727625, 22: 0.910320558731268, 23: 0.9967711618096655, 24: 0.44404974665543817, 25: 0.3464944534705235, 26: 0.576028086893188, 27: 0.6768626772490913, 28: 0.7667147667894887, 29: 0.9955096543499555, 30: 0.5238543129825075, 31: 0.5599085722160164, 32: 0.5901538606669944, 33: 0.7256886984161701, 34: 0.7374665066266308, 35: 0.9697439931165255, 36: 0.535805675734872, 37: 0.5592452891944351, 38: 0.5756647374752185, 39: 0.6436426017849102, 40: 0.8336044711449767, 41: 0.9589390716358468,

You can also visualize the results in the plantGL scene

In [7]:
SceneWidget(lighting.plantGL_sensors(light=True), 
            position=(-2.5, -2.5, 0.0), 
            size_display=(600, 400), 
            plane=True, 
            size_world = 10, 
            axes_helper=True)

AttributeError: 'LightVegeManager' object has no attribute '_LightVegeManager__sensors_plantgl'

In [11]:
SceneWidget(lighting.plantGL_sensors(light=True) + lighting.plantGL_nolight(), 
            position=(-2.5, -2.5, 0.0), 
            size_display=(600, 400), 
            plane=True, 
            size_world = 10, 
            axes_helper=True)

SceneWidget(axes_helper=True, scenes=[{'id': 'A5C5KpsvluofwnlhgGz2Qa4N7', 'data': b'x\xda\x8d\x9c\t\x98\x16\xc…

### Other parameters

In additional features, you can activate the debug mode in CARIBU, which describe the internal steps.

In [6]:
caribu_args = { "debug" : True }

lighting = LightVegeManager(lightmodel="caribu", lightmodel_parameters=caribu_args)
lighting.build(geometry=[(0., 0., 0.), (1., 0., 0.), (1., 1., 1.)])

energy = 500.
hour = 15
day = 264
lighting.run(energy=energy, hour=hour, day=day)

Prepare scene 1
done


FileNotFoundError: [Errno 2] No such file or directory: './caribuscene_2672400716080\\cscene.can'

You can also use the soilmesh option and get the lighting hitting the soil. The method `soilenergy` get you access to its result.

In [5]:
caribu_args = { "soil mesh" : True }

lighting = LightVegeManager(lightmodel="caribu", lightmodel_parameters=caribu_args)
lighting.build(geometry=[(0., 0., 0.), (1., 0., 0.), (1., 1., 1.)])

energy = 500.
hour = 15
day = 264
lighting.run(energy=energy, hour=hour, day=day)

print(lighting.soilenergy)

{'Qi': 0.6750540873627096, 'Einc': 0.6750540873627096}


## RATP

This is the complete parameters you can provide with CARIBU: 


```python
ratp_args = {
                # Grid specifications
                "voxel size" : [dx, dy, dz],
                "voxel size" : "dynamic",
                
                "full grid" : bool,

                "origin" : [xorigin, yorigin, zorigin],
                "origin" : [xorigin, yorigin],

                "number voxels" : [nx, ny, nz],
                "grid slicing" : "ground = 0."
                "tesselation level" : int

                # Leaf angle distribution
                "angle distrib algo" : "compute global",
                "angle distrib algo" : "compute voxel",
                "angle distrib algo" : "file",

                "nb angle classes" : int,
                "angle distrib file" : filepath,

                # Vegetation type
                "soil reflectance" : [reflectance_band0, reflectance_band1, ...],
                "reflectance coefficients" : [reflectance_band0, reflectance_band1, ...],
                "mu" : [mu_scene0, mu_scene1, ...]
            }
```

### Leaf angle distribution

Leaf angle distribution can be generated in 3 ways:
- from a file, one distribution per specy
- global and dynamically, it generates a distribution from a triangles mesh for each specy
- per voxel and dynamically, it generates a distribution from the triangles located in each voxel

In [2]:
# random triangles
nb_triangles = 50
spheresize = (1., 0.3) # vertices of triangles are the sphere surface
worldsize = (0., 5.)
triangles = [random_triangle_generator(worldsize=worldsize, spheresize=spheresize) for i in range(nb_triangles)]

#### File

You need the flag `"file"` and to specify the file path.

In [15]:
filepath = os.path.join(os.path.dirname(os.path.abspath("")), "data", "luzerne_angle_distrib.data")
ratp_parameters = { "angle distrib algo" : "file", "angle distrib file" : filepath }

# initialize the instance
lighting = LightVegeManager(lightmodel="ratp", lightmodel_parameters=ratp_parameters)

# build the scene
lighting.build(geometry=triangles)

print(lighting.leafangledistribution)

{'global': [[0.1382, 0.1664, 0.1972, 0.1925, 0.1507, 0.0903, 0.0425, 0.0172, 0.005]]}


#### Global distribution

You need the flag `"compute global"` and to specify the number of angle classes you need.

In [3]:
ratp_parameters = { "angle distrib algo" : "compute global", "nb angle classes" : 9 }

# initialize the instance
lighting = LightVegeManager(lightmodel="ratp", lightmodel_parameters=ratp_parameters)

# build the scene
lighting.build(geometry=triangles)

print(lighting.leafangledistribution)

{'global': [[0.0, 0.01403472520994376, 0.14553389876181141, 0.13870880399511626, 0.17786938035077418, 0.06842434118299212, 0.09608726894836216, 0.20983153542452182, 0.14951004612647875]]}


#### Local distribution

You need the flag `"compute voxel"` and to specify the number of angle classes you need. You will get one distribution for each voxel of your grid and each specy.

In [6]:
ratp_parameters = { 
                    "voxel size" : [1., 1., 1.],
                    "angle distrib algo" : "compute voxel", 
                    "nb angle classes" : 9
                    }

# initialize the instance
lighting = LightVegeManager(lightmodel="ratp", lightmodel_parameters=ratp_parameters)

# build the scene
lighting.build(geometry=triangles)

print("Global")
print(lighting.leafangledistribution["global"])
print("\n\n Local")
for a in lighting.leafangledistribution["voxel"]:
    print(a[0])

Global
[[0.009674637223767685, 0.06468549489243054, 0.10705660994810652, 0.10196105003699654, 0.07575759861707439, 0.3044721953841436, 0.06446975879644407, 0.14828246866341796, 0.12364018643761852]]


 Local
[0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0.         0.         0.55805526 0.44194474 0.         0.
 0.         0.         0.        ]
[0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0.         0.         0.41000514 0.58999486 0.         0.
 0.         0.         0.        ]
[0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0.

For visualization of the situation

In [7]:
SceneWidget(lighting.plantGL_nolight(printtriangles=True, printvoxels=True), 
            position=(-2.5, -2.5, 0.0), 
            size_display=(600, 400), 
            plane=True, 
            size_world = 10, 
            axes_helper=True)

SceneWidget(axes_helper=True, scenes=[{'id': 'Rp6rgkWOhuO88XZzffdSscV0V', 'data': b'x\xda\x8d\x9b{\x9c\x8d\xd5…

### Triangles tesselation in a grid

You can reduce the error while transferring a triangle mesh to a voxel mesh by subdividing triangles across multiple voxels.

<div>
<img src="attachment:tesselation_voxels.png" width="500"/>
</div>

You only need to precise how many times you want to subdivide the triangles.

In [21]:
ratp_parameters = { "voxel size" : [1., 1., 1.], "tesselation level" : 7 }

# initialize the instance
lighting = LightVegeManager(lightmodel="ratp", lightmodel_parameters=ratp_parameters)

# build the scene
lighting.build(geometry=triangles)

SceneWidget(lighting.plantGL_nolight(printtriangles=True, printvoxels=True), 
            position=(-2.5, -2.5, 0.0), 
            size_display=(600, 400), 
            plane=True, 
            size_world = 10, 
            axes_helper=True)

SceneWidget(axes_helper=True, scenes=[{'id': 'w9mu1jQPBqZrSzDk4B45B9Wav', 'data': b'x\xda\x94}\x07\x9c^E\xd5~\…

### Other parameters

By default, the number of voxels is dynamically computed following the voxel size and mesh limits, but you can force its number. 

Voxel size can also be dynamically computed and is based on 3 times the longest triangle.